In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from joblib import Parallel, delayed

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms

In [ ]:
images_dir = '/kaggle/input/nhapmoncv/data/images'
classes = [d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))]

label_map = {cls: idx for idx, cls in enumerate(classes)}

In [ ]:
data = []
for cls in classes:
    cls_folder = os.path.join(images_dir, cls)
    for fname in os.listdir(cls_folder):
        if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
            file_path = os.path.join(cls_folder, fname)
            label = label_map[cls]
            data.append((file_path, label))

classes = [d.split("-")[-1] for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))]
label_map = {cls: idx for idx, cls in enumerate(classes)}


In [ ]:
df = pd.DataFrame(data, columns=['filepath', 'label'])
print(df.head())
print("Number of images:", len(df))
print("Number of classes:", len(classes))

label_map = {v: k for k, v in label_map.items()}
df["breed"] = df["label"].map(label_map)

print(df.head())

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

alexnet = models.alexnet(pretrained=True)

feature_extractor = nn.Sequential(
    alexnet.features,
    alexnet.avgpool,
    nn.Flatten()
)
feature_extractor.to(device)
feature_extractor.eval()


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
def compute_alexnet_features(img_path):
        img = Image.open(img_path).convert("RGB")
        img_t = transform(img).unsqueeze(0).to(device)
        with torch.no_grad():
            features = feature_extractor(img_t)
        return features.cpu().numpy().flatten()

In [ ]:
features_list = Parallel(n_jobs=4, backend='loky')(
    delayed(compute_alexnet_features)(p) for p in df["filepath"]
)

valid_mask = [f is not None for f in features_list]
X = np.vstack([f for f in features_list if f is not None])
y = df.loc[valid_mask, "label"].values

print(f"AlexNet features extracted for {len(X)} of {len(df)} images")

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)

print("Accuracy (AlexNet features):", accuracy_score(y_test, y_pred) * 100)